In [2]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
def load_data(train_folder,test_folder):
    train_file_paths = []
    train_labels = []
    
    test_file_paths = []
    test_labels = []
    
    #Getting the train data
    train_sub_folders = os.listdir(train_folder)
    for train_sub_folder in train_sub_folders:
        train_files = os.listdir(os.path.join(train_folder,train_sub_folder))
        for train_file in train_files:
            train_file_paths.append(os.path.join(os.path.join(train_folder,train_sub_folder),train_file))
            train_labels.append(train_sub_folder)
    
    
    #Getting the test data
    test_sub_folders = os.listdir(test_folder)
    for test_sub_folder in test_sub_folders:
        test_files = os.listdir(os.path.join(test_folder,test_sub_folder))
        for test_file in test_files:
            test_file_paths.append(os.path.join(os.path.join(test_folder,test_sub_folder),test_file))
            test_labels.append(test_sub_folder)
            
    #Shuffling the data and making the cross validation data and making pandas data frame
    Ftrain_series = pd.Series(name = 'filepath',data = train_file_paths)
    Strain_series = pd.Series(name = 'label', data = train_labels)
    train_df = pd.concat([Ftrain_series,Strain_series], axis = 1)
    
    Ftest_series = pd.Series(name = 'filepath',data = test_file_paths)
    Stest_series = pd.Series(name = 'label', data = test_labels)
    temp_df = pd.concat([Ftest_series,Stest_series], axis = 1)
    
    test_df, cross_val_df = train_test_split(temp_df, train_size = 0.5, shuffle = True)
    
    return train_df, test_df, cross_val_df
    

In [4]:
train_df, test_df, val_df = load_data(test_folder='/kaggle/input/brain-tumor-mri-dataset/Testing', train_folder= '/kaggle/input/brain-tumor-mri-dataset/Training')

In [5]:
temp_train_gen = ImageDataGenerator(rescale=1./255)
temp_test_gen = ImageDataGenerator(rescale=1./255)
temp_val_gen = ImageDataGenerator(rescale=1./255)

train_gen = temp_train_gen.flow_from_dataframe(train_df, directory=None, x_col='filepath', y_col='label', weight_col=None, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='png', subset=None, interpolation='nearest', validate_filenames=True)
test_gen = temp_test_gen.flow_from_dataframe(test_df, directory=None, x_col='filepath', y_col='label', weight_col=None, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='png', subset=None, interpolation='nearest', validate_filenames=True)
val_gen = temp_val_gen.flow_from_dataframe(val_df, directory=None, x_col='filepath', y_col='label', weight_col=None, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='png', subset=None, interpolation='nearest', validate_filenames=True)
#flow_from_dataframe(dataframe, directory=None, x_col='filename', y_col='class', weight_col=None, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='png', subset=None, interpolation='nearest', validate_filenames=True)


Found 5712 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.


In [6]:
model = Sequential()

img_shape = (256, 256, 3)

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu", input_shape= img_shape))
model.add(MaxPooling2D())
    
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D())
    
model.add(Flatten())
    
model.add(Dense(64,activation = "relu"))
model.add(Dense(32,activation = "relu"))
model.add(Dense(4, activation = "softmax"))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 262144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    16,777,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,818,212 (64.16 MB)

 Trainable params: 16,818,212 (64.16 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
fit = model.fit(train_gen, epochs=8, validation_data=val_gen,shuffle= False)

Epoch 1/8


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


179/179 ━━━━━━━━━━━━━━━━━━━━ 622s 3s/step - accuracy: 0.5748 - loss: 1.1266 - val_accuracy: 0.8308 - val_loss: 0.5003
Epoch 2/8
179/179 ━━━━━━━━━━━━━━━━━━━━ 619s 3s/step - accuracy: 0.8917 - loss: 0.3086 - val_accuracy: 0.9131 - val_loss: 0.2456
Epoch 3/8
179/179 ━━━━━━━━━━━━━━━━━━━━ 622s 3s/step - accuracy: 0.9511 - loss: 0.1533 - val_accuracy: 0.9314 - val_loss: 0.1648
Epoch 4/8
179/179 ━━━━━━━━━━━━━━━━━━━━ 617s 3s/step - accuracy: 0.9790 - loss: 0.0772 - val_accuracy: 0.9451 - val_loss: 0.1742
Epoch 5/8
179/179 ━━━━━━━━━━━━━━━━━━━━ 623s 3s/step - accuracy: 0.9917 - loss: 0.0404 - val_accuracy: 0.9588 - val_loss: 0.1256
Epoch 6/8
179/179 ━━━━━━━━━━━━━━━━━━━━ 614s 3s/step - accuracy: 0.9972 - loss: 0.0186 - val_accuracy: 0.9634 - val_loss: 0.1294
Epoch 7/8
179/179 ━━━━━━━━━━━━━━━━━━━━ 633s 3s/step - accuracy: 0.9985 - loss: 0.0108 - val_accuracy: 0.9573 - val_loss: 0.1476
Epoch 8/8
164/179 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.9999 - loss: 0.0058

In [ ]:
model.save("brain_tumor_classifier.h5")
#The story of stuff